# Studio: Working with Databases in Python

For today's studio, we will be using the [TV Shows dataset](https://www.kaggle.com/ruchi798/tv-shows-on-netflix-prime-video-hulu-and-disney) from Kaggle. We have already downloaded the CSV for you.

You will also be using the watchlist you just created to figure out which streaming services contain the shows that you want to watch next so you can decide which one is worth the money to you.

As you complete the different tasks in the studio, you may choose between using Pandas or SQL. Remember that during the prep work, we learned that one is oftentimes more efficient at certain tasks than the other, so choose wisely!

## My Watchlist

If you would like, please use this space to make note of your watchlist by editing the text cell.

Watchlist:

Supernatural

Seinfield

Modern Family

Amazing Races

Next Level Chef

One Tree Hill

Gossip Girl

Gilmore Girls

Fire Country

Chicago Fire

## Database Setup

To start with, import the necessary libraries and create a dataframe from the provided CSV. Print the info out for the dataframe. After that, you may drop the column called `Unnamed: 0` and rename any columns with spaces in the names or unusual characters such as `"+"`. Print out the info for the dataframe again to ensure that your changes were effective.

In [91]:
import sqlite3 as sl
import pandas as pd

df = pd.read_csv("tv_shows.csv")

df = df.drop(columns=['Unnamed: 0'])

df['Age'] = df["Age"].str.replace('+', '')
df = df.rename(columns ={'Rotten Tomatoes': 'Rotten_Tomatoes', 'Prime Video': 'Prime_Video', 'Disney+': 'Disney'} )

print(df.head())

   ID             Title  Year Age    IMDb Rotten_Tomatoes  Netflix  Hulu  \
0   1      Breaking Bad  2008  18  9.4/10         100/100        1     0   
1   2   Stranger Things  2016  16  8.7/10          96/100        1     0   
2   3   Attack on Titan  2013  18  9.0/10          95/100        1     1   
3   4  Better Call Saul  2015  18  8.8/10          94/100        1     0   
4   5              Dark  2017  16  8.8/10          93/100        1     0   

   Prime_Video  Disney  Type  
0            0       0     1  
1            0       0     1  
2            0       0     1  
3            0       0     1  
4            0       0     1  


C:\Users\kap19\AppData\Local\Temp\ipykernel_19068\1975674892.py:8: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['Age'] = df["Age"].str.replace('+', '')


With your dataframe at the ready, create a new database called `tv`. Add a new table to your database called `shows` using the data in the dataframe. 

In [95]:
con = sl.connect("tv.db")

df.to_sql('shows1', con)

5368

With your new table and database set up, print out the top 20 records in the `shows` table.

In [62]:
print(df.head(20))

    ID                       Title  Year Age    IMDb Rotten_Tomatoes  Netflix  \
0    1                Breaking Bad  2008  18  9.4/10         100/100        1   
1    2             Stranger Things  2016  16  8.7/10          96/100        1   
2    3             Attack on Titan  2013  18  9.0/10          95/100        1   
3    4            Better Call Saul  2015  18  8.8/10          94/100        1   
4    5                        Dark  2017  16  8.8/10          93/100        1   
5    6  Avatar: The Last Airbender  2005   7  9.3/10          93/100        1   
6    7              Peaky Blinders  2013  18  8.8/10          93/100        1   
7    8            The Walking Dead  2010  18  8.2/10          93/100        1   
8    9                Black Mirror  2011  18  8.8/10          92/100        1   
9   10          The Queen's Gambit  2020  18  8.6/10          92/100        1   
10  11                  Mindhunter  2017  18  8.6/10          90/100        1   
11  12                   Com

Now add a table that includes an id number and the titles on your watchlist.

In [98]:
with con:
    con.execute('''
    CREATE TABLE watchlist_top_10 (
        title TEXT);
    ''')

sql = 'INSERT INTO watchlist_top_10 (title) VALUES (?)'
data = [
    ('Supernatural'),
    ('Seinfield'),
    ('Modern Family'),
    ('Amazing Races'),
    ('Next Level Chef'),
    ('One Tree Hill'),
    ('Gossip Girl'),
    ('Gilmore Girls'),
    ('Fire Country'),
    ('Chicago Fire')
]

OperationalError: table watchlist_top_10 already exists

In [99]:
con.executemany(sql, data)
data = con.execute("SELECT * FROM watchlist_top_10")
for row in data:
    print(row)

ProgrammingError: Incorrect number of bindings supplied. The current statement uses 1, and there are 12 supplied.

## Working with the Data

Using either Pandas or SQL, determine how many shows are on each streaming service and what the mean is for each streaming service. 

**Note**: You may notice that the mean represents the percentage of shows in the dataframe that are on each streaming service. Take a moment to think about why that is.

In [92]:
print(df.describe())

print(df["Netflix"].value_counts())
print(df["Hulu"].value_counts())
print(df["Prime_Video"].value_counts())
print(df["Disney"].value_counts())

                ID         Year      Netflix         Hulu  Prime_Video  \
count  5368.000000  5368.000000  5368.000000  5368.000000  5368.000000   
mean   2814.945790  2012.628726     0.367176     0.301975     0.341095   
std    1672.385727    10.142813     0.482080     0.459157     0.474122   
min       1.000000  1904.000000     0.000000     0.000000     0.000000   
25%    1345.750000  2011.000000     0.000000     0.000000     0.000000   
50%    2788.000000  2016.000000     0.000000     0.000000     0.000000   
75%    4308.250000  2018.000000     1.000000     1.000000     1.000000   
max    5717.000000  2021.000000     1.000000     1.000000     1.000000   

            Disney    Type  
count  5368.000000  5368.0  
mean      0.065387     1.0  
std       0.247231     0.0  
min       0.000000     1.0  
25%       0.000000     1.0  
50%       0.000000     1.0  
75%       0.000000     1.0  
max       1.000000     1.0  
0    3397
1    1971
Name: Netflix, dtype: int64
0    3747
1    1621
Name

Join your watchlist data with the shows data to determine which streaming services your watchlist shows are on and make a new table in your database using the joined data. Print out the data in your joined table to see what shows on your watchlist on in the original dataset. With the joined data, determine the percentage of your watchlist shows that are on each streaming service and how many of your watchlist shows are on each streaming service.

In [97]:
df= pd.read_sql("""SELECT watchlist_top_10.title, shows1.Netflix, shows1.Hulu, shows1.Prime_Video, shows1.Disney
FROM watchlist_top_10 JOIN shows1 on shows1.Title == watchlist_top_10.title;
""", con)


print(df)

Empty DataFrame
Columns: [title, Netflix, Hulu, Prime_Video, Disney]
Index: []


## Results

Now that you have done your analysis, make note of the answers to the following questions by editing the text cell:

1. Was every show on your watchlist in the Kaggle dataset? Do you have any ideas as to why a show might not have been present?
2. Did you include a show or shows in your watchlist that is exclusive to one of the platforms? How might that have impacted your analysis?
2. Which streaming service(s) offered the most shows on your watchlist? Which streaming service(s) offered the least?
3. Based on the shows you want to watch and the results of your analysis, is there a streaming service you think would be a good fit for you?